In [ ]:
# 라이브러리 임포트
from sentence_transformers import SentenceTransformer
import mysql.connector
import chromadb
from chromadb import PersistentClient
import time
from datetime import datetime
import shutil
import os

chroma_path = "./chroma_db"
if os.path.exists(chroma_path):
    shutil.rmtree(chroma_path)
    print("✅ 기존 ChromaDB 삭제 완료!")
else:
    print("✅ 초기화할 ChromaDB가 없습니다.")


# 임베딩 모델 로딩
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# MySQL 연결
conn = mysql.connector.connect(
    host="192.168.14.47",
    user="dongdong",
    password="20250517",
    database="ai_re",
    connection_timeout=3600     # 1시간까지 MySQL 연결 유지
)
cursor = conn.cursor()

# ChromaDB 연결 및 컬렉션 생성
client = PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="recipes_kr_sbert")

✅ 초기화할 ChromaDB가 없습니다.


In [ ]:
# 중복 키 체크용 세트 (re_name + re_ingredient)
existing_data = collection.get()
existing_keys = set()
for doc, meta in zip(existing_data["documents"], existing_data["metadatas"]):
    key = f"{meta.get('name', '')}|{meta.get('ingredient', '')}"
    existing_keys.add(key)
print(f"기존 저장된 키 수: {len(existing_keys)}")

✅ 기존 저장된 키 수: 0


In [9]:
# 배치 적재 파라미터 설정
batch_size = 500
offset = 0
total_inserted = 0

# 총 개수 파악 (예상 시간 계산용)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM recipe")
total_rows = cursor.fetchone()[0]
cursor.close()
estimated_batches = total_rows // batch_size + 1
print(f"\n🧲 전체 레시피 수: {total_rows}개 → 총 예상 루프 수: {estimated_batches}회")
start_time = time.time()

# 로그 파일 열기
log_file_path = "chroma_insert_log.txt"
with open(log_file_path, "a", encoding="utf-8") as log:

    # 루프 수행
    while True:
        cursor = conn.cursor()  # 하위에서 컨널팅 오류 바인드 체크
        query = f"""
            SELECT id, name, instruction, role, ingredient, category, inputrecipe, portnum, level, timenum, style
            FROM recipe
            LIMIT {batch_size} OFFSET {offset}
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        cursor.close()

        if not rows:
            print("\n✅ 모든 레시피 생입 완료!")
            log.write(f"[{datetime.now().strftime('%H:%M:%S')}] ✅ 모든 레시피 생입 완료!\n")
            break

        docs, metas, ids = [], [], []

        for row in rows:
            id, name, instruction, role, ingredient, category, inputrecipe, portnum, level, timenum, style = row      
            text = f"{id}: {name}, 재료: {inputrecipe}, 카테고리: {category}, 조리방법: {instruction}, 유형: {style}, 주재료: {ingredient}"
            key = f"{name}|{inputrecipe}"

            if key in existing_keys:
                continue  # 중복 건너떡

            meta = {
                "id": id,
                "name": name,
                "inputrecipe": inputrecipe,
                "category": category,
                "level": level,
                "cook_time": timenum,
                "portnum": portnum,
                "role": role, 
                "instructions": instruction,
                "style": style,
                "ingredient": ingredient,
            }

            docs.append(text)
            metas.append(meta)
            ids.append(f"rec_{id}")
            existing_keys.add(key)

        # 임벤드 및 생입
        if docs:
            embeddings = model.encode(docs).tolist()
            collection.add(
                documents=docs,
                metadatas=metas,
                embeddings=embeddings,
                ids=ids
            )
            total_inserted += len(docs)

            elapsed = time.time() - start_time
            eta = (elapsed / (offset // batch_size + 1)) * (estimated_batches - (offset // batch_size + 1)) / 60

            log_line = f"[{datetime.now().strftime('%H:%M:%S')}] 현재까지 생입된 레시피 수: {total_inserted} | 예상 남은 시간: {eta:.1f}분\n"
            print(log_line.strip())
            log.write(log_line)

            if total_inserted % 10000 < batch_size:
                log.write("[CHECKPOINT] ---- 저장 시점 도달 ----\n")

        offset += batch_size


🧲 전체 레시피 수: 200개 → 총 예상 루프 수: 1회
[16:03:39] 현재까지 생입된 레시피 수: 200 | 예상 남은 시간: 0.0분

✅ 모든 레시피 생입 완료!


In [ ]:
# 연결 종료
cursor.close()
conn.close()

In [3]:
from sentence_transformers import SentenceTransformer
import mysql.connector
import chromadb
from chromadb import PersistentClient
import time
from datetime import datetime
import shutil
import os

# 임베딩 모델 로딩
query = '두부'

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

client = PersistentClient(path="C:/Users/Admin/workspace/market_service/vectordb/chroma_db")
collection = client.get_collection(name="recipes_kr_sbert")
# 컬렉션 불러오기
collection = client.get_collection(name="recipes_kr_sbert")

# 사용자 쿼리 → 임베딩
query_embedding = model.encode([query]).tolist()

# 유사 문서 검색
result = collection.query(query_embeddings=query_embedding, n_results=10)

# 레시피 출력
result = result["metadatas"][0]

In [4]:
result

[{'portnum': 5,
  'name': '두부배추들깨국',
  'level': '초급',
  'style': '한식',
  'id': 7030943,
  'category': '국/탕',
  'inputrecipe': '[재료] 육수 1 L | 알배기배추 60g | 두부 150g | 대파 80g | 표고버섯 60g | 다진마늘 15.0g | 들깨가루 60.0g | 액젓 30.0g | 국간장 30.0g | 무 100g',
  'role': '일상',
  'cook_time': 30,
  'instructions': '끓이기',
  'ingredient': '채소류'},
 {'instructions': '삶기',
  'cook_time': 15,
  'level': '초급',
  'name': '열무물국수',
  'style': '한식',
  'role': '일상',
  'inputrecipe': '[재료] 열무김치 100g | 국수소면 100g | 삶은계란 25g | 채소육수 400g | 통깨 2.5g | 설탕 15.0g ',
  'category': '면/만두',
  'id': 7033719,
  'ingredient': '밀가루',
  'portnum': 1},
 {'instructions': '끓이기',
  'style': '한식',
  'level': '아무나',
  'role': '일상',
  'cook_time': 30,
  'ingredient': '소고기',
  'inputrecipe': '[재료] 소고기 600g | 무 800g | 두부 300g | 국간장 22.5g',
  'portnum': 4,
  'category': '국/탕',
  'name': '맑은소고기무국',
  'id': 7019954},
 {'category': '국/탕',
  'inputrecipe': '[재료] 아욱 300g | 두부 150g [양념] 집된장 15.0g | 쌀뜨물 1 L |  밥새우 150g | 청양고추 20g | 다진마늘 45.0g',
  'instr

# 완성된 ChromaDB를 fastAPI로 공유하기
- http://localhost:8000/docs
- http://192.168.0.68:8000/docs

# FastAPI 서버 실행하기 (cmd에서 실행)
uvicorn chroma_api:app --reload --host 0.0.0.0 --port 8000
